### CS424

Prof. Götz Pfeiffer<br />
School of Mathematics, Statistics and Applied Mathematics<br />
NUI Galway

# Lecture 13: Migrations

It is not unusual that the structure  of a database changes in the process of
developing the application.  In fact, in incremental development it is almost
impossible  to  avoid  modifications  in   the  database  structure,  as  the
application  grows, and  the specifications  potentially change.   Migrations
provide a way to perform these  structural changes with little effort, and to
keep them under control.

* Migrations provide a convenient way to **alter the database** in a structured and organized manner.
* `Rails` migrations are **database agnostic**.
* `Rails` **tracks** which migrations have already been run (and which ones still need to be run).
* To apply outstanding migrations, update your source and run `rake db:migrate`.
* Migrations are **`ruby` programs** (using SQL only as a very last resort).
* As programs on file, migrations can be version controlled.

## The Shape of a Migration

Migrations are used to create new database tables, for example a table called `products`:
```ruby
class CreateProducts < ActiveRecord::Migration
  def change
    create_table :products do |t|
      t.string :title
      t.text :description
      t.string :image_url
      t.decimal :price, precision: 8, scale: 2

      t.timestamps null: false
    end
  end
end
```

This migration is a `ruby` class, named after its purpose, which defines
a single method `change`.

The `change` method, when called, creates a new table `products` with 2 columns `title` and `image_url` of type `string`, a column `description` of type `text`
and a column `price` of type `decimal`.

A **primary key** column `id` is automatically added;
this is the default primary key for all `Rails` models.

The `timestamps` method adds two
**timestamp** columns `created_at` and `updated_at`,
which are automatically managed.

This particular migration was automatically generated, as part of the
output of the 
```bash
rails g scaffold products
```
command.
It sits in its own file in the `db/migrations` folder, where it
can be found and modified if necessary, before it is applied.

In the above example, the generated migration was modified to specify
the exact format of the `price` column as 8 significant digits
with 2 digits after the decimal point.

Migrations are  usually dealt  with in a such a two-step process.   First, a
migration is  created (as a  `ruby` class  in its own  file).  Secondly,
after  optionally  modifying  the  generated file,  the  migration  is
applied.

## Column Types

`rails` supports the following database column types, independent of the actual database used by the application:

* `:binary`
* `:boolean`
* `:date`
* `:datetime`
* `:decimal`
* `:float`
* `:integer`
* `:primary_key`
* `:string`
* `:text`
* `:time`
* `:timestamp`

## Common Data Definition Tasks

Migrations are meant to be invertible (i.e., undoable) operations.
The command
```bash
rake db:rollback
```
can be used to undo the latest migration.

There are methods that perform common data definition tasks in a database
independent way:

* `add_column`
* `remove_column`
* `rename_column`
* `change_column`

* `create_table`
* `drop_table`
* `change_table`

* `add_index`
* `remove_index`



For most tasks, there is an obvious opposite task and `rails` can automatically
invert a migration, when asked to do so.  E.g, to undo a `create_table` task,
one applies the `drop_table` task to the same table name.

Alternatively, or in less obvious situations, a migration can implement
a pair of methods, `up` and `down`,
where `up` specifies how to perform the required transformations,
and `down` how to revert them.

## Creating Migrations Automatically

Migrations can be created through a variety of commands.

* `rails g scaffold ...` and `rails g model ...` will generate
migrations for creating the database tables for new models.

* `rails g migration name` will generate a named migration file,
which usually needs to be modified to suit its intended purpose.

Migrations are frequently used to add/remove/rename columns to/from/in a
database table.
Rails understands migration names structured like

* `rails g migration add_name_to_table name:type` or

* `rails g migration RemoveNameFromTable name:type`

in `under_line` or `CamelCase` convention, and automatically generates files with
the correct `up`, `down` or `change` functions.

For example, the command
```bash
rails g migration add_quantity_to_line_items quantity:integer
```
generates a file named `20171017141031_add_quantity_to_line_items.rb`
with content:
```ruby
class AddQuantityToLineItems < ActiveRecord::Migration
  def change
    add_column :line_items, :quantity, :integer
  end
end
```
The corresponding migration `remove_quantity_from_line_items`
would work without specifying the type of the `quantity` column,
but that information is useful for inferring the opposite task.

Like all migration files, the name of this file starts with a **timestamp**
prefix `20171017141031_` which `rake` uses to manage a timeline of 
(old and new) migrations.

There are several commands for applying or undoing migrations.

* `rake db:migrate` runs the `change` or `up` methods of
all those migrations (in the `db/migrations` folder) that have
not been applied yet.

* `rake db:rollback` undoes the last migration.

* `rake db:rollback STEP=3` undoes the last 3 migrations.

* `rake db:migrate VERSION=20171011120000` restores the structure
of the database at a given point in time.

* `rake db:reset` drops the entire database and then recreates it
(structure only, content will be lost forever).

The technical term for the structure of a database is **schema**.
The file `db/schema.rb` is a Ruby program that describes the
current structure of the database (as the result of running
migrations).  It can be used to start a new instance of the app
(without goinng through the individual migrations).

## Other Uses

Other uses of migrations include systematic modification of bad data,
inital population of new fields, .... In these cases `Rails` cannot guess the
programmers intentions and the programmer has to fill in the details
(in the generated file) themselves.

## Undoing Prematurely Applied Migrations

If a migration has been applied in error:

* rollback
* edit/modify migration as necessary
* run the migration again.
